# Figure 1
- schematic
- spectrogram for NSD and SD
- lfp snapshots for sleep classification


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from neuropy.utils import signal_process
import subjects
import pandas as pd

sd_sessions = subjects.sd.allsess[:-1]
nsd_sessions = subjects.nsd.allsess
sd_sess = sd_sessions[3]
nsd_sess = nsd_sessions[3]
print(sd_sessions)
print(nsd_sessions)

[ProcessData(RatJ_Day1_2019-05-31_03-55-36.xml), ProcessData(RatK_Day1_2019-08-06_03-44-01.xml), ProcessData(RatN_Day1_2019-10-09_03-52-32.xml), ProcessData(RatS_Day3SD_2020-11-29_07-53-30.xml), ProcessData(RatR_Day2SD_2021-05-23_09-07-18.xml)]
[ProcessData(RatJ_Day2_2019-06-02_03-59-19.xml), ProcessData(RatK_Day2_2019-08-08_04-00-00.xml), ProcessData(RatN_Day2_2019-10-11_03-58-54.xml), ProcessData(RatS-Day2NSD-2020-11-27_10-22-29.xml), ProcessData(RatR_Day1NSD_2021-05-13_08-41-31.xml)]


## Calculating spectrogram

In [21]:
nsd_signal = nsd_sess.eegfile.get_signal(63)
nsd_spect = signal_process.SpectrogramBands(nsd_signal, window=5, overlap=2)

sd_signal = sd_sess.eegfile.get_signal(113)
sd_spect = signal_process.SpectrogramBands(sd_signal, window=5, overlap=2)

## Sleep fraction and normalized firing rate within
- normalized firing rate or mean firing rate across population?
- across all states and also in individual states

In [4]:
sd_df = pd.DataFrame()
for sub, sess in enumerate(sd_sessions):
    post = sess.paradigm["post"]
    rec_duration = sess.eegfile.duration
    bins = np.arange(0, post[1], 600)
    neurons = sess.neurons.get_neuron_type('pyr')

    pre_neur =neurons.time_slice(0,post[0])
    pre_binned_spk = pre_neur.get_binned_spiketrains(bin_size=900)
    frate_pre =np.mean(pre_binned_spk.spike_counts/pre_binned_spk.bin_size,axis=0)
    n_bins_pre = pre_binned_spk.n_bins

    post_neur =neurons.time_slice(post[0],post[1])
    post_binned_spk = post_neur.get_binned_spiketrains(bin_size=900)
    frate_post = np.mean(post_binned_spk.spike_counts/post_binned_spk.bin_size,axis=0)
    n_bins_post = post_binned_spk.n_bins
    
    sub_df = pd.DataFrame({'bins':np.arange(-n_bins_pre,n_bins_post),'frate':np.concatenate((frate_pre,frate_post))})
    sd_df = sd_df.append(sub_df)


sd_frate = sd_df.groupby('bins')
sd_frate_mean = sd_frate.mean().frate
sd_frate_sem = sd_frate.sem(ddof=0).frate

nsd_df = pd.DataFrame()
for sub, sess in enumerate(nsd_sessions[:-1]):
    post = sess.paradigm["post"]
    rec_duration = sess.eegfile.duration
    bins = np.arange(0, post[1], 600)
    neurons = sess.neurons.get_neuron_type('pyr')

    pre_neur =neurons.time_slice(0,post[0])
    pre_binned_spk = pre_neur.get_binned_spiketrains(bin_size=900)
    frate_pre =np.mean(pre_binned_spk.spike_counts/pre_binned_spk.bin_size,axis=0)
    n_bins_pre = pre_binned_spk.n_bins

    post_neur =neurons.time_slice(post[0],post[1])
    post_binned_spk = post_neur.get_binned_spiketrains(bin_size=900)
    frate_post = np.mean(post_binned_spk.spike_counts/post_binned_spk.bin_size,axis=0)
    n_bins_post = post_binned_spk.n_bins
    
    sub_df = pd.DataFrame({'bins':np.arange(-n_bins_pre,n_bins_post),'frate':np.concatenate((frate_pre,frate_post))})
    nsd_df = nsd_df.append(sub_df)


nsd_frate = nsd_df.groupby('bins')
nsd_frate_mean = nsd_frate.mean().frate
nsd_frate_sem = nsd_frate.sem(ddof=0).frate


## Plotting

In [5]:
%matplotlib widget
from matplotlib.patches import Rectangle

figure = plotting.Fig()
fig,gs = figure.draw(num=1,grid=(4,3))

# gs_sd = figure.subplot2grid(gs[0,:],grid=(5,1))

# ax_sd_spect = plt.subplot(gs_sd[1:])
# plotting.plot_spectrogram(sd_spect.sxx,sd_spect.time,sd_spect.freq,ax=ax_sd_spect)
# ax_sd_spect.vlines([sd_sess.paradigm['pre'][1],sd_sess.paradigm['maze1'][1],sd_sess.paradigm['sd'][1],sd_sess.paradigm['post'][1]],0,30,ls='--',color='w',lw=2)

# for ep in sd_sess.paradigm.to_dataframe().itertuples():
#     ax_sd_spect.text(ep.start+ep.duration/2,18,ep.label,color='w',rotation=45,ha='center',fontweight='bold')

# ax_sd_hyp = plt.subplot(gs_sd[0],sharex=ax_sd_spect)
# plotting.plot_hypnogram(sd_sess.brainstates,ax=ax_sd_hyp)

# figure.panel_label(ax_sd_hyp,'a')



# # ------ panel b-----------
# gs_nsd = figure.subplot2grid(gs[1,:],grid=(5,1))

# ax_nsd_spect = plt.subplot(gs_nsd[1:])
# plotting.plot_spectrogram(nsd_spect.sxx,nsd_spect.time,nsd_spect.freq,ax=ax_nsd_spect)
# ax_nsd_spect.vlines([nsd_sess.paradigm['pre'][1],nsd_sess.paradigm['maze1'][1],nsd_sess.paradigm['post'][1]],0,30,ls='--',color='w',lw=2)

# for ep in nsd_sess.paradigm.to_dataframe().itertuples():
#     ax_nsd_spect.text(ep.start+ep.duration/2,18,ep.label,color='w',rotation=45,ha='center',fontweight='bold')

# ax_nsd_hyp = plt.subplot(gs_nsd[0],sharex=ax_nsd_spect)
# plotting.plot_hypnogram(nsd_sess.brainstates,ax=ax_nsd_hyp)

# figure.panel_label(ax_nsd_hyp,'b')

#------ panel c --------
ax_frate = plt.subplot(gs[2,:2])
ax_frate.errorbar(sd_frate_mean.index.values,sd_frate_mean,sd_frate_sem,ecolor='k')
ax_frate.errorbar(nsd_frate_mean.index.values,nsd_frate_mean,nsd_frate_sem,ecolor='r')
ax_frate.set_xlabel('ZT Bin')
ax_frate.set_ylabel('Firing rate')
# ax_frate.plot(sd_frate_mean)
# ax_frate.fill_between(sd_frate_mean+sd_frate_sem)
# for ax_ind in range(2):
#     ax = plt.subplot(gs[ax_ind])
#     ax.plot([1,5],[2,2],color='w',zorder=1,alpha=0)

#     start = 1
#     for i, (l,w,col) in enumerate(zip(["PRE",'MAZE','POST'],[5,3,10],['#849ad7','#7ac792','#849ad7'])):
#         ax.add_patch(Rectangle((start, 1.5), w,2 ,facecolor=col))
#         ax.text(start+w/2,0.5,l,rotation=45,ha='center')
#         start+=w+0.1
#     ax.axis('off')

# ax.add_patch(Rectangle((start-w,2),w/2,1,facecolor='#de8cdc'))
# ax.text(start-w+0.5,2.3,'SD')

# #----- spectrograms--------

# ax_nsd_spect = plt.subplot(gs[2,:])
# plotting.plot_spectrogram(nsd_spect.sxx,nsd_spect.time,nsd_spect.freq,ax=ax_nsd_spect)

# ax_sd_spect = plt.subplot(gs[3,:])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Firing rate')